![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 3. Spark NLP Pretrained Models v2.6.3

Spark NLP offers the following pre-trained models in 26 languages and all you need to do is to load the pre-trained model into your disk by specifying the model name and then configuring the model parameters as per your use case and dataset. Then you will not need to worry about training a new model from scratch and will be able to enjoy the pre-trained SOTA algorithms directly applied to your own data with transform().

In the official documentation, you can find detailed information regarding how these models are trained by using which algorithms and datasets.

https://github.com/JohnSnowLabs/spark-nlp-models

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.226.30:7077 
 AppName 
 Databricks Shell

## LemmatizerModel

In [0]:
!wget -q -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv

In [0]:
%fs ls "file:/databricks/driver"

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/news_category_test.csv,news_category_test.csv,1504408
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/university_basketball.sqlite,university_basketball.sqlite,20480
file:/databricks/driver/derby.log,derby.log,717
file:/databricks/driver/schema_converted.json,schema_converted.json,2322


In [0]:
#dbutils.fs.cp("file:/databricks/driver/news_category_test.csv", "dbfs:/")

Out[3]: True

In [0]:
import pyspark.sql.functions as F

news_df = spark.read\
                .option("header", "true")\
                .csv("news_category_test.csv")\
                .withColumnRenamed("description", "text")

news_df.show(truncate=50)

+--------+--------------------------------------------------+
category| text|
+--------+--------------------------------------------------+
Business|Unions representing workers at Turner Newall ...|
Sci/Tech| TORONTO, Canada A second team of rocketeers...|
Sci/Tech| A company founded by a chemistry researcher at...|
Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|
Sci/Tech| Southern California's smog fighting agency wen...|
Sci/Tech|"The British Department for Education and Skill...|
Sci/Tech|"confessed author of the Netsky and Sasser viru...|
Sci/Tech|\\FOAF/LOAF and bloom filters have a lot of in...|
Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|
Sci/Tech|In its first two years, the UK's dedicated card...|
Sci/Tech| A group of technology companies including Tex...|
Sci/Tech| Apple Computer Inc.&lt;AAPL.O&gt; on Tuesday ...|
Sci/Tech| Free Record Shop, a Dutch music retail chain,...|
Sci/Tech|A giant 100km colony of ants which has been di...|
Sci/Tech| "Dolphin groups, or ""pods""|
Sci/Tech|Tyrannosaurus rex achieved its massive size due...|
Sci/Tech| Scientists have discovered irregular lumps be...|
Sci/Tech| ESAs Mars Express has relayed pictures from o...|
Sci/Tech|When did life begin? One evidential clue stems ...|
Sci/Tech|update Earnings per share rise compared with a ...|
+--------+--------------------------------------------------+
only showing top 20 rows

In [0]:
lemmatizer = LemmatizerModel.pretrained('lemma_antbnc', 'en') \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \


'''
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("file:/databricks/driver/AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")
'''

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][OK!]
Out[5]: '\nlemmatizer = Lemmatizer() .setInputCols(["token"]) .setOutputCol("lemma") .setDictionary("file:/databricks/driver/AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")\n'

In [0]:
!cd ~/cache_pretrained && ls -l


In [0]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)


In [0]:
result = pipelineModel.transform(news_df)

result.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
category| text| document| token| stem| lemma|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[token, 0, 5, un...|[[token, 0, 5, Un...|
Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[token, 1, 7, to...|[[token, 1, 7, TO...|
Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[token, 1, 1, a,...|[[token, 1, 1, A,...|
Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|[[token, 1, 4, it...|[[token, 1, 4, It...|
Sci/Tech| Southern Califor...|[[document, 0, 17...|[[token, 1, 8, So...|[[token, 1, 8, so...|[[token, 1, 8, So...|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
result.select('token.result','lemma.result').show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[Unions, representing, workers, at, Turner, Newall, say, they, are, ', disappointed, ', after, ta...|[Unions, represent, worker, at, Turner, Newall, say, they, be, ', disappointed, ', after, talk, w...|
[TORONTO, ,, Canada, A, second, team, of, rocketeers, competing, for, the, #36;10, million, Ansar...|[TORONTO, ,, Canada, A, second, team, of, rocketeer, compete, for, the, #36;10, million, Ansari, ...|
[A, company, founded, by, a, chemistry, researcher, at, the, University, of, Louisville, won, a, ...|[A, company, founded, by, a, chemistry, researcher, at, the, University, of, Louisville, win, a, ...|
[It's, barely, dawn, when, Mike, Fitzpatrick, starts, his, shift, with, a, blur, of, colorful, ma...|[It's, barely, dawn, when, Mike, Fitzpatrick, start, he, shift, with, a, blur, of, colorful, map,...|
[Southern, California's, smog, fighting, agency, went, after, emissions, of, the, bovine, variety...|[Southern, California's, smog, fight, agency, go, after, emission, of, the, bovine, variety, Frid...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
only showing top 5 rows

## PerceptronModel (POS - Part of speech tags)

In [0]:
pos = PerceptronModel.pretrained("pos_anc", 'en')\
      .setInputCols("document", "token")\
      .setOutputCol("pos")

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[ | ][ / ][ — ][OK!]

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)


In [0]:
result = pipelineModel.transform(news_df)

result.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
category| text| document| token| stem| lemma| pos|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[token, 0, 5, un...|[[token, 0, 5, Un...|[[pos, 0, 5, NNP,...|
Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[token, 1, 7, to...|[[token, 1, 7, TO...|[[pos, 1, 7, NNP,...|
Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[token, 1, 1, a,...|[[token, 1, 1, A,...|[[pos, 1, 1, DT, ...|
Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|[[token, 1, 4, it...|[[token, 1, 4, It...|[[pos, 1, 4, NNP,...|
Sci/Tech| Southern Califor...|[[document, 0, 17...|[[token, 1, 8, So...|[[token, 1, 8, so...|[[token, 1, 8, So...|[[pos, 1, 8, NNP,...|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
result.select('token.result','pos.result').show(5, truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[Unions, representing, workers, at, Turner, Newall, say, they, are, ', disappointed, ', after, ta...| [NNP, VBG, NNS, IN, NNP, NNP, VBP, PRP, VBP, POS, JJ, POS, IN, NNS, IN, NN, NN, NN, NNP, NNP, .]|
[TORONTO, ,, Canada, A, second, team, of, rocketeers, competing, for, the, #36;10, million, Ansar...|[NNP, ,, NNP, DT, JJ, NN, IN, NNS, VBG, IN, DT, NN, CD, NNP, NNP, NNP, ,, DT, NN, IN, RB, JJ, JJ,...|
[A, company, founded, by, a, chemistry, researcher, at, the, University, of, Louisville, won, a, ...|[DT, NN, VBN, IN, DT, NN, NN, IN, DT, NNP, IN, NNP, VBD, DT, NN, TO, VB, DT, NN, IN, VBG, JJR, NN...|
[It's, barely, dawn, when, Mike, Fitzpatrick, starts, his, shift, with, a, blur, of, colorful, ma...|[NNP, RB, NN, WRB, NNP, NNP, VBZ, PRP$, NN, IN, DT, NN, IN, JJ, NNS, ,, NNS, CC, JJ, NNS, ,, CC, ...|
[Southern, California's, smog, fighting, agency, went, after, emissions, of, the, bovine, variety...|[NNP, NNP, NN, VBG, NN, VBD, IN, NNS, IN, DT, NN, NN, NNP, ,, VBG, DT, NN, JJ, NNS, TO, VB, NN, N...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
only showing top 5 rows

In [0]:
# applying this pipeline to top 100 rows and then converting to Pandas

result = pipelineModel.transform(news_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'token.begin', 'token.end', 'stem.result',  'lemma.result', 'pos.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']").alias("stem"),
        F.expr("cols['4']").alias("lemma"),
        F.expr("cols['5']").alias("pos")).toPandas()

result_df.head(10) 

,token,begin,end,stem,lemma,pos
0,Unions,0,5,union,Unions,NNP
1,representing,7,18,repres,represent,VBG
2,workers,20,26,worker,worker,NNS
3,at,28,29,at,at,IN
4,Turner,31,36,turner,Turner,NNP
5,Newall,40,45,newal,Newall,NNP
6,say,47,49,sai,say,VBP
7,they,51,54,thei,they,PRP
8,are,56,58,ar,be,VBP
9,',60,60,',',POS


In [0]:
# same in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('Unions representing workers at Turner   Newall say they are disappointed after talks with stricken parent firm Federal Mogul.')

list(zip(light_result['token'], light_result['stem'], light_result['lemma'], light_result['pos']))

Out[19]: [('Unions', 'union', 'Unions', 'NNP'),
 ('representing', 'repres', 'represent', 'VBG'),
 ('workers', 'worker', 'worker', 'NNS'),
 ('at', 'at', 'at', 'IN'),
 ('Turner', 'turner', 'Turner', 'NNP'),
 ('Newall', 'newal', 'Newall', 'NNP'),
 ('say', 'sai', 'say', 'VBP'),
 ('they', 'thei', 'they', 'PRP'),
 ('are', 'ar', 'be', 'VBP'),
 ('disappointed', 'disappoint', 'disappointed', 'VBN'),
 ('after', 'after', 'after', 'IN'),
 ('talks', 'talk', 'talk', 'NNS'),
 ('with', 'with', 'with', 'IN'),
 ('stricken', 'stricken', 'stricken', 'NN'),
 ('parent', 'parent', 'parent', 'NN'),
 ('firm', 'firm', 'firm', 'NN'),
 ('Federal', 'feder', 'Federal', 'NNP'),
 ('Mogul', 'mogul', 'Mogul', 'NNP'),
 ('.', '.', '.', '.')]

## Chunker

Meaningful phrase matching

This annotator matches a pattern of part-of-speech tags in order to return meaningful phrases from document

Output type: Chunk

Input types: Document, POS

Functions:

`setRegexParsers(patterns)`: A list of regex patterns to match chunks, for example: Array(“‹DT›?‹JJ›*‹NN›

`addRegexParser(patterns)`: adds a pattern to the current list of chunk patterns, for example: “‹DT›?‹JJ›*‹NN›”

In [0]:
# applying POS chunker to find a custom pattern

chunker = Chunker()\
    .setInputCols(["document", "pos"])\
    .setOutputCol("chunk")\
    .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

# NNP: Proper Noun
# NN: COmmon Noun
# DT: Determinator (e.g. the)
# JJ: Adjective

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 chunker
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

result.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
category| text| document| token| stem| lemma| pos| chunk|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[token, 0, 5, un...|[[token, 0, 5, Un...|[[pos, 0, 5, NNP,...|[[chunk, 0, 5, Un...|
Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[token, 1, 7, to...|[[token, 1, 7, TO...|[[pos, 1, 7, NNP,...|[[chunk, 1, 7, TO...|
Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[token, 1, 1, a,...|[[token, 1, 1, A,...|[[pos, 1, 1, DT, ...|[[chunk, 52, 61, ...|
Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|[[token, 1, 4, it...|[[token, 1, 4, It...|[[pos, 1, 4, NNP,...|[[chunk, 1, 4, It...|
Sci/Tech| Southern Califor...|[[document, 0, 17...|[[token, 1, 8, So...|[[token, 1, 8, so...|[[token, 1, 8, So...|[[pos, 1, 8, NNP,...|[[chunk, 1, 21, S...|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:

result_df = result.select(F.explode(F.arrays_zip('chunk.result', 'chunk.begin',  'chunk.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,chunk,begin,end
0,Unions,0,5
1,Turner Newall,31,45
2,Federal Mogul,113,125
3,stricken,92,99
4,parent,101,106
5,firm,108,111
6,TORONTO,1,7
7,Canada,10,15
8,Ansari X Prize,82,95
9,A second team,20,32


## Dependency Parser

In [0]:
dep_parser = DependencyParserModel.pretrained('dependency_conllu')\
        .setInputCols(["document", "pos", "token"])\
        .setOutputCol("dependency")

dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
typed_dep_parser = TypedDependencyParserModel.pretrained('dependency_typed_conllu')\
        .setInputCols(["token", "pos", "dependency"])\
        .setOutputCol("dependency_type")

dependency_typed_conllu download started this may take some time.
Approximate size to download 257.4 KB
[ | ][ / ][ — ][OK!]

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 dep_parser,
 typed_dep_parser
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'token.begin',  'token.end', 'dependency.result', 'dependency_type.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']").alias("dependency"),
        F.expr("cols['4']").alias("dependency_type")).toPandas()

result_df.head(10)

,chunk,begin,end,dependency,dependency_type
0,Unions,0,5,ROOT,root
1,representing,7,18,workers,amod
2,workers,20,26,Unions,flat
3,at,28,29,Turner,case
4,Turner,31,36,workers,flat
5,Newall,40,45,say,nsubj
6,say,47,49,Unions,parataxis
7,they,51,54,disappointed,nsubj
8,are,56,58,disappointed,nsubj
9,',60,60,disappointed,case


## StopWordsCleaner

`stopwords_fr`, `stopwords_de`, `stopwords_en`, `stopwords_it`, `stopwords_af` .... over 40 languages

In [0]:

stopwords_cleaner = StopWordsCleaner.pretrained('stopwords_en','en')\
        .setInputCols("token")\
        .setOutputCol("cleanTokens")\
        .setCaseSensitive(False)



stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[ | ][OK!]

In [0]:
# we can also get the list of stopwords 

stopwords_cleaner_es = StopWordsCleaner.pretrained('stopwords_es','es')\
        .setInputCols("token")\
        .setOutputCol("cleanTokens")\
        .setCaseSensitive(False)

stopwords_cleaner_es.getStopWords()[:10]

stopwords_es download started this may take some time.
Approximate size to download 2.2 KB
[ | ][OK!]
Out[28]: ['a',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'adrede',
 'ahi',
 'ahí',
 'ahora',
 'al']

In [0]:
token_assembler = TokenAssembler() \
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("clean_text")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stopwords_cleaner,
 token_assembler

 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

# same in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('Peter Parker is a nice person and a friend of mine.')

light_result['clean_text']


Out[29]: ['Peter Parker nice person friend mine.']

## SpellChecker

### Norvig Spell Checker

This annotator retrieves tokens and makes corrections automatically if not found in an English dictionary

In [0]:
spell_checker_norvig = NorvigSweetingModel.pretrained('spellcheck_norvig')\
        .setInputCols("token")\
        .setOutputCol("corrected")


spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[ | ][ / ][OK!]

In [0]:
from pyspark.sql.types import StringType

text_list = ['Peter Parker is a nice persn and lives in New York.', 'Bruce Wayne is also a nice guy and lives in Gotham City.']

spark_df = spark.createDataFrame(text_list, StringType()).toDF("text")

spark_df.show(truncate=80)

+--------------------------------------------------------+
 text|
+--------------------------------------------------------+
 Peter Parker is a nice persn and lives in New York.|
Bruce Wayne is also a nice guy and lives in Gotham City.|
+--------------------------------------------------------+

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 spell_checker_norvig
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(spark_df)


In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip('token.result', 'corrected.result', 'stem.result',  'lemma.result', 'pos.result')).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("corrected"),
                          F.expr("cols['2']").alias("stem"),
                          F.expr("cols['3']").alias("lemma"),
                          F.expr("cols['4']").alias("pos")).toPandas()

result_df.head(10)

,token,corrected,stem,lemma,pos
0,Peter,Peter,peter,Peter,NNP
1,Parker,Parker,parker,Parker,NNP
2,is,is,i,be,VBZ
3,a,a,a,a,DT
4,nice,nice,nice,nice,JJ
5,persn,person,persn,persn,NN
6,and,and,and,and,CC
7,lives,lives,live,life,NNS
8,in,in,in,in,IN
9,New,New,new,New,NNP


In [0]:
# same in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('The patint has pain and headace')

list(zip(light_result['token'], light_result['corrected']))


Out[35]: [('The', 'The'),
 ('patint', 'patient'),
 ('has', 'has'),
 ('pain', 'pain'),
 ('and', 'and'),
 ('headace', 'headache')]

### Context SpellChecker

The idea for this annotator is to have a flexible, configurable and "re-usable by parts" model.

Flexibility is the ability to accommodate different use cases for spell checking like OCR text, keyboard-input text, ASR text, and general spelling problems due to orthographic errors.

We say this is a configurable annotator, as you can adapt it yourself to different use cases avoiding re-training as much as possible.

Spell Checking at three levels: The final ranking of a correction sequence is affected by three things,

Different correction candidates for each word - **word level**.

The surrounding text of each word, i.e. it's context - **sentence level**.

The relative cost of different correction candidates according to the edit operations at the character level it requires - **subword level**.

In [0]:

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_dl')\
    .setInputCols("token")\
    .setOutputCol("checked")

finisher = Finisher()\
    .setInputCols("checked")

pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel,
    finisher
  ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

sc_model = pipeline.fit(empty_ds)
lp = LightPipeline(sc_model)

spellcheck_dl download started this may take some time.
Approximate size to download 112 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
lp.annotate("Plaese alliow me tao introdduce myhelf, I am a man of waelth und tiaste")


Out[37]: {'checked': ['Please',
 'allow',
 'me',
 'to',
 'introduce',
 'myself',
 ',',
 'I',
 'am',
 'a',
 'man',
 'of',
 'wealth',
 'and',
 'taste']}

In [0]:
examples = ['We will go to swimming if the ueather is nice.',\
    "I have a black ueather jacket, so nice.",\
    "I introduce you to my sister, she is called ueather."]

spark_df = spark.createDataFrame(examples, StringType()).toDF("text")

results = sc_model.transform(spark_df)

results.show(truncate=False)


## Language Detector

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")


languageDetector = LanguageDetectorDL.pretrained('ld_wiki_20', 'xx')\
      .setInputCols("document")\
      .setOutputCol("language")\
      .setThreshold(0.5)\
      .setCoalesceSentences(True)

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 languageDetector
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)


ld_wiki_20 download started this may take some time.
Approximate size to download 3 MB
[ | ][ / ][ — ][ \ ][OK!]

In [0]:
light_model = LightPipeline(pipelineModel)


In [0]:
text_en = "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist. He is best known as the co-founder of Microsoft Corporation. During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014."

text_de = 'Als Sebastian Thrun 2007 bei Google anfing, an selbstfahrenden Autos zu arbeiten, nahmen ihn nur wenige Leute außerhalb des Unternehmens ernst.'

text_es = "La historia del procesamiento del lenguaje natural generalmente comenzó en la década de 1950, aunque se puede encontrar trabajo de períodos anteriores. En 1950, Alan Turing publicó un artículo titulado 'Maquinaria de computación e inteligencia' que proponía lo que ahora se llama la prueba de Turing como criterio de inteligencia"

text_it = "Geoffrey Everest Hinton è uno psicologo cognitivo e uno scienziato informatico canadese inglese, noto soprattutto per il suo lavoro sulle reti neurali artificiali. Dal 2013 divide il suo tempo lavorando per Google e l'Università di Toronto. Nel 2017 è stato cofondatore ed è diventato Chief Scientific Advisor del Vector Institute di Toronto."



In [0]:
light_model.annotate(text_de)['language']

Out[43]: ['de']

In [0]:
light_model.fullAnnotate(text_es)[0]['language']

Out[44]: [Annotation(language, 0, 328, es, {'hr': '3.0852183E-7', 'fr': '2.3359653E-5', 'uk': '3.7443638E-7', 'pt': '3.811586E-5', 'cs': '7.113755E-6', 'el': '1.1423608E-5', 'it': '3.415775E-5', 'bg': '1.9164734E-6', 'de': '3.5754451E-6', 'ru': '1.998705E-5', 'sv': '1.3846361E-6', 'tr': '3.8109105E-5', 'en': '4.0024934E-5', 'sk': '2.7228594E-5', 'es': '0.9997111', 'fi': '6.467608E-7', 'ro': '6.1194587E-6', 'no': '2.988667E-6', 'sentence': '0', 'hu': '1.4394581E-5', 'pl': '1.7670265E-5'})]

## Embeddings

### Word Embeddings (Glove)

In [0]:
glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")
    

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 glove_embeddings
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(3))


In [0]:
result.select('embeddings.embeddings').take(1)

Out[13]: [Row(embeddings=[[0.7186499834060669, 0.8075399994850159, -1.1786999702453613, 0.271450012922287, -0.48833000659942627, -0.1893800050020218, -1.1789000034332275, 0.1783600002527237, -0.21995000541210175, -0.7215999960899353, -0.5354200005531311, -0.7507500052452087, 0.41176000237464905, 0.18951000273227692, -0.5093500018119812, -0.735010027885437, 0.33823999762535095, 0.3451400101184845, -0.16735999286174774, 0.130390003323555, 1.1055999994277954, 0.5508400201797485, -0.01406800001859665, -0.5322800278663635, -0.23598000407218933, -1.0496000051498413, -0.6549199819564819, -0.17997999489307404, 0.10341999679803848, -0.315530002117157, 0.31481000781059265, 0.8438799977302551, 0.305400013923645, -1.158400058746338, 0.047777000814676285, -0.04945499822497368, 0.7908999919891357, -0.22619999945163727, -0.6294800043106079, 0.3116999864578247, -0.7399600148200989, 0.1283400058746338, 0.10723999887704849, 0.3257099986076355, -0.2687300145626068, -0.70169997215271, 0.16669000685214996, 0.3308899998664856, -0.04666899889707565, 0.13434000313282013, -0.3903599977493286, 0.35043999552726746, -0.6380500197410583, 1.0003999471664429, -0.1405400037765503, -0.6695600152015686, 0.29333001375198364, -0.04130199924111366, 1.5181000232696533, 0.9282100200653076, -0.0594559982419014, -0.8433300256729126, -0.21630999445915222, -0.364329993724823, 0.0641229972243309, -0.0647289976477623, -0.32370999455451965, 0.32034000754356384, 0.9140099883079529, -0.8716099858283997, 0.14883999526500702, -0.4030100107192993, -1.2542999982833862, -0.7203999757766724, 0.22737999260425568, -0.27138999104499817, 0.25005999207496643, -1.037500023841858, -1.256500005722046, -0.12337999790906906, -0.09109599888324738, -0.980970025062561, 0.20313000679016113, -0.4651400148868561, -1.8416999578475952, -0.6694599986076355, 0.013848000206053257, 1.0679999589920044, -0.08525200188159943, -0.5746700167655945, 0.5118200182914734, -0.6081799864768982, -1.0676000118255615, 0.2016499936580658, -0.13197000324726105, 0.41545000672340393, -0.17709000408649445, 0.5877199769020081, 1.126099944114685, -0.5484700202941895], [0.2567099928855896, 0.30035001039505005, -0.1800599992275238, 0.4666599929332733, 0.985010027885437, 0.2320999950170517, -0.349590003490448, 0.26996999979019165, -0.9966700077056885, -0.43404000997543335, -0.17270000278949738, -1.267799973487854, 0.6458699703216553, -0.08055999875068665, -0.10688000172376633, -0.6668199896812439, 0.12387000024318695, 0.4363200068473816, -0.42590999603271484, 0.23579999804496765, 0.291949987411499, 0.22596000134944916, 0.5508999824523926, -0.42719000577926636, -0.38655999302864075, -0.827970027923584, 0.5543400049209595, -0.13357999920845032, 0.38161998987197876, -0.3511199951171875, 0.688979983329773, 0.2200700044631958, 0.2505500018596649, -0.6122499704360962, 0.2902100086212158, -0.33941999077796936, 0.4240800142288208, 0.7613300085067749, 0.08363699913024902, 2.965199928439688e-05, -0.482369989156723, -0.5486500263214111, 0.3570300042629242, 0.11528000235557556, -0.02913600020110607, 0.11303000152111053, 0.06495899707078934, -0.2025900036096573, -0.5543599724769592, 0.05228099972009659, 0.2602599859237671, -0.19032999873161316, 0.44394999742507935, 0.4488700032234192, -0.5525699853897095, -0.8007299900054932, -0.23316000401973724, -0.5227100253105164, 1.0738999843597412, 0.7515199780464172, -0.043306998908519745, 0.14406999945640564, 0.01055500004440546, 0.06772799789905548, 0.10349000245332718, -0.10739000141620636, 0.30557000637054443, 0.9226899743080139, 0.3269599974155426, 0.1467200070619583, 0.1668899953365326, 0.416269987821579, -0.7763100266456604, 0.2741599977016449, 0.5012400150299072, -0.05087300017476082, 0.053603000938892365, -0.6662099957466125, -1.0152000188827515, -0.46022000908851624, -0.1068200021982193, 0.11057999730110168, -0.12228000164031982, -0.3601199984550476, -0.5704500079154968, -0.567110002040863, -0.45089998841285706, 0.011590000241994858, 0.7907800078392029, -0.19756999611854553, 0.1193400025

In [0]:
result = pipelineModel.transform(news_df.limit(3))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("embeddings"))

result_df.show(10, truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
 token| embeddings|
+------------+----------------------------------------------------------------------------------------------------+
 Unions|[0.71865, 0.80754, -1.1787, 0.27145, -0.48833, -0.18938, -1.1789, 0.17836, -0.21995, -0.7216, -0....|
representing|[0.25671, 0.30035, -0.18006, 0.46666, 0.98501, 0.2321, -0.34959, 0.26997, -0.99667, -0.43404, -0....|
 workers|[0.50592, 0.71717, -0.67236, -0.32112, -0.58285, -0.47977, -0.50243, 0.60594, 0.25709, 0.03974, -...|
 at|[0.1766, 0.093851, 0.24351, 0.44313, -0.39037, 0.12524, -0.19918, 0.59855, -0.82035, 0.28006, 0.5...|
 Turner|[0.51634, -0.37186, -0.21776, -1.0115, 0.4014, -0.4841, 0.36274, -0.2952, -0.42258, -0.62844, 0.6...|
 Newall|[-0.38857, -1.1449, -0.41737, -0.31969, -0.16546, -0.7044, 0.12875, -0.26047, 0.072844, -0.13314,...|
 say|[-0.091682, 0.58105, 0.40477, -0.41979, -0.85111, -0.28719, -0.41949, -0.10424, 0.45317, -0.09907...|
 they|[-0.07954, 0.30171, 0.079516, -0.74662, -0.67879, 0.35029, -0.19754, 0.4929, 0.14162, -0.23789, 0...|
 are|[-0.51533, 0.83186, 0.22457, -0.73865, 0.18718, 0.26021, -0.42564, 0.67121, -0.31084, -0.61275, 0...|
 '|[-0.34562, -0.24993, 0.58678, -0.89119, -1.0954, -0.45078, -0.074549, -0.44779, -0.38492, -0.4923...|
+------------+----------------------------------------------------------------------------------------------------+
only showing top 10 rows

### Elmo Embeddings

Computes contextualized word representations using character-based word representations and bidirectional LSTMs.

It can work with 4 different pooling layer options: `word_emb`, 
`lstm_outputs1`, `lstm_outputs2`, or `elmo`

In [0]:
elmo_embeddings = ElmoEmbeddings.pretrained('elmo')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")\
          .setPoolingLayer('elmo')# default --> elmo
    

elmo download started this may take some time.
Approximate size to download 334.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 elmo_embeddings
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("elmo_embeddings"))

result_df.show(truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
 token| elmo_embeddings|
+------------+----------------------------------------------------------------------------------------------------+
 Unions|[-0.90887433, -0.30120444, 0.4379642, -0.5704447, 0.47783926, -0.11942903, 0.007673677, 0.3835698...|
representing|[0.13965528, 0.22222611, -0.039318148, 0.016958907, 0.7448398, 0.34160846, -0.3798289, -0.2486626...|
 workers|[-0.46730536, -0.30479306, -0.026426047, -0.3242935, 0.75354195, -0.18502724, -0.12659091, -0.464...|
 at|[-0.25518382, -0.22426198, -0.3707175, -0.6011193, 0.06384373, -0.48253897, 0.43195236, 0.5064739...|
 Turner|[-1.2854298, -0.08228696, -0.19137493, -0.42979652, -0.2290245, 0.11305164, 0.28118274, -0.632974...|
 Newall|[-0.15922368, -0.40697074, -0.26923797, -0.43109438, 0.029521652, -0.34705538, -0.17874797, 0.088...|
 say|[0.040825367, 0.79921997, 0.020270282, -0.51678205, 0.08984972, -0.5492892, -0.091029376, -0.0168...|
 they|[-1.1243963, -0.010669652, -0.09746781, -0.0038198605, 0.88272464, -0.89751065, 0.2576731, 0.5404...|
 are|[-0.7048798, -0.34280774, -0.07719928, -0.49473476, 0.014530212, -0.12821783, 0.29851735, 0.52826...|
 '|[-0.5458977, -0.2953822, 0.023100264, -0.019431174, 0.25288737, 0.009414971, 0.17324883, -0.03460...|
disappointed|[-0.6758905, -0.20069833, 0.83521044, -0.50575554, 0.6561525, -0.6367345, -0.16388406, -0.9043924...|
 '|[-0.2575528, 0.30305576, 0.5622954, 0.30021662, 0.5401405, -0.5633521, 0.24766469, -0.579903, 0.8...|
 after|[-0.2023893, 0.24767333, -0.23489517, -0.32719916, 0.4426178, -0.9367191, 0.21963787, -0.12545657...|
 talks|[-0.17850737, 0.25418758, 0.08238873, -0.24434857, 0.17956313, -1.212021, 0.07875112, 0.3737101, ...|
 with|[0.52579856, -0.15434292, -0.11828968, -0.13308679, -0.057415567, 0.0146114975, 0.27711594, 0.126...|
 stricken|[0.42973414, -0.1418663, 0.06929128, -0.23070805, 0.045775652, -0.10202133, 0.68093747, 0.7641754...|
 parent|[0.44972163, -0.15741444, 0.16139536, -0.06740509, 0.61041915, 0.18980025, -0.19194509, 1.1387346...|
 firm|[0.23465869, -0.27139944, 0.22050472, -0.044672966, 0.44497526, -0.49274576, -0.042227462, 0.1930...|
 Federal|[-0.9843948, 1.3339462, 0.39078698, -0.50403816, 0.71886545, -0.2895932, 0.7570523, 0.17455813, 0...|
 Mogul|[-0.4634575, -0.12920582, 0.11130282, -0.26346898, 0.9858172, -1.0185294, -0.060575366, 0.3652845...|
+------------+----------------------------------------------------------------------------------------------------+
only showing top 20 rows

### Bert Embeddings

BERT (Bidirectional Encoder Representations from Transformers) provides dense vector representations for natural language by using a deep, pre-trained neural network with the Transformer architecture

It can work with 3 different pooling layer options: `0`, 
`-1`, or `-2`

In [0]:
bert_embeddings = BertEmbeddings.pretrained('bert_base_cased')\
                                .setInputCols(["document", "token"])\
                                .setOutputCol("embeddings")

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 bert_embeddings
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("bert_embeddings"))

result_df.show(truncate=100)

+------------+----------------------------------------------------------------------------------------------------+
 token| bert_embeddings|
+------------+----------------------------------------------------------------------------------------------------+
 Unions|[-0.46760347, -0.16900869, -0.0397435, 0.25002235, -0.26480156, -0.22864302, 0.04333419, 0.129555...|
representing|[-0.4491169, 0.34213677, -0.533321, 0.43395427, -0.43488488, -0.37696314, -0.17551038, 0.28233862...|
 workers|[0.3941771, -0.39944446, 0.22336268, 0.45902923, 0.07521692, -0.23951805, -0.33352423, 0.33830568...|
 at|[0.37566483, 0.1904456, -0.4414509, 0.15454617, 0.42388952, -0.2798142, 0.22211401, 0.37631083, -...|
 Turner|[0.031253777, -0.24268252, -0.37302524, 0.20491852, 0.09712863, -0.0035196617, -0.07763222, 0.616...|
 Newall|[-0.018507447, -0.56981826, 0.25690654, 0.42174372, -0.44619006, -0.09330324, 0.38020694, -0.3418...|
 say|[-0.15611628, -0.34698448, -0.024027396, 0.051422358, 0.33272487, -0.08601738, 0.09307597, 0.2963...|
 they|[-0.12951396, 0.23536104, -0.4629938, -0.31093043, 0.6970239, -0.19754411, 0.65694517, 0.45188028...|
 are|[-0.21750164, -6.1142235E-4, -0.47941098, 0.24785668, 0.6717297, -0.34232774, 0.25475043, 0.41317...|
 '|[0.030203603, -0.18581253, -0.9198395, -0.11049655, 0.6191022, -0.7184093, 0.05288955, 0.17506175...|
disappointed|[0.14849614, -0.1436468, -0.22086014, -0.49341258, 0.06751902, -0.50204784, -0.0024874313, 0.1146...|
 '|[0.27277243, 0.5575879, -0.56302404, 0.1510146, 0.6297077, -0.14008194, 0.3129914, 0.37835667, 0....|
 after|[-0.007154852, -0.15318695, -0.23245253, 0.28341222, 0.85852134, -0.69557077, 0.09175559, 0.20845...|
 talks|[0.060111888, 0.3689915, 0.031831536, 0.46082887, -0.33121324, -0.60188556, 0.07556082, -0.238439...|
 with|[0.36686024, -0.17995173, -0.6140605, 0.14136675, 0.15484226, -0.44141674, -0.11372744, 0.1155695...|
 stricken|[0.2385819, -0.060876593, -0.64635456, -0.13124478, -0.04843765, -0.15930258, -0.19889228, 0.1511...|
 parent|[0.12695584, 0.1615971, -0.66906375, 0.09284499, 0.11506555, -0.3074776, -0.20355414, 0.50433046,...|
 firm|[-0.21374601, 0.22260907, -0.20400135, -0.2843647, 0.42999366, -0.28247964, -0.09254529, 0.616147...|
 Federal|[0.13525738, -0.22775318, -0.09767661, 0.0128184315, 0.1276173, -0.19585928, 0.13670735, 0.639977...|
 Mogul|[0.75738245, -0.5795739, 0.09359647, 0.06380835, 0.47653258, -0.54582304, 0.10456904, 0.47946584,...|
+------------+----------------------------------------------------------------------------------------------------+
only showing top 20 rows

### XlnetEmbeddings

Computes contextualized word representations using combination of Autoregressive Language Model and Permutation Language Model

In [0]:
xlnet_embeddings = XlnetEmbeddings.pretrained('xlnet_base_cased')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 xlnet_embeddings
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'embeddings.embeddings')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("bert_embeddings"))

result_df.show(truncate=100)

xlnet_base_cased download started this may take some time.
Approximate size to download 415.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
+------------+----------------------------------------------------------------------------------------------------+
 token| bert_embeddings|
+------------+----------------------------------------------------------------------------------------------------+
 Unions|[-0.85942733, -0.5574012, 1.3173033, -1.1469395, -0.08702204, 1.1876523, -0.08872084, -1.0941846,...|
representing|[-0.12686437, -1.5573617, 1.1400528, 0.7962866, 1.9238433, -1.8473315, -0.51913106, -0.6651284, -...|
 workers|[-1.1859174, -0.84681255, 1.6014686, -0.87219125, 0.75299305, 0.4634779, 1.2401329, -0.8114535, -...|
 at|[0.040925328, -1.0951927, -1.7422688, 0.01532897, -1.0510317, -0.67333513, 1.2163581, -1.8911494,...|
 Turner|[0.18314204, 0.8931678, 0.51512086, 0.22869428, 1.1063783, -0.11106407, -0.400284, -1.6873496, -1...|
 Newall|[-1.077733, 0.053559832, 0.23795721, -0.69574696, -0.6543494, 0.8698385, -1.2967412, -1.5392374, ...|
 say|[-0.44416705, 0.8378129, -0.08979457, 0.09618027, -2.3527064, -0.0109050935, 1.1900387, -1.272123...|
 they|[0.52291477, -0.42804104, -1.4166158, 0.9568587, -0.060344007, -0.34999862, 0.9018336, -1.6067948...|
 are|[0.17354317, -1.2573661, -0.8466841, -0.5577354, -0.60430825, -0.9621323, 0.112761915, -0.7255048...|
 '|[-1.2489276, -1.243969, 0.5209835, -0.66235745, 0.9791761, 0.16123521, -1.5246601, -0.9585142, -0...|
disappointed|[1.2122922, -1.4161003, -1.8436407, 0.25174138, 0.9680219, 1.2591715, 1.0421705, -2.1084476, -1.4...|
 '|[-1.2489276, -1.243969, 0.5209835, -0.66235745, 0.9791761, 0.16123521, -1.5246601, -0.9585142, -0...|
 after|[-1.0372891, -1.2394243, -0.18529208, -0.4958639, 0.08627766, 0.32589906, 0.010660787, 0.34127977...|
 talks|[-0.49822465, 0.18008982, 0.0205384, 0.5141163, -0.6948355, 0.5348742, -1.6602131, -0.27333173, -...|
 with|[0.79053605, -1.2425677, -1.4570476, -0.3674038, -0.9684857, 0.17137419, 1.1612104, -0.6689384, -...|
 stricken|[0.029818598, -0.8363373, 1.3775382, -1.4370009, 0.24844041, 1.8644342, 0.3191535, 0.07758907, -1...|
 parent|[-0.3173735, 0.23081473, 0.41221341, 0.7424053, -1.109319, -0.2656174, -0.8515507, -0.69435835, -...|
 firm|[-0.87550503, -0.70715594, 0.44855574, -1.5679716, 0.31086436, -2.3357399, -0.12809141, -0.371086...|
 Federal|[-1.2244954, -1.8920977, -0.5046513, -0.88803875, 0.3308447, 1.2062479, 0.59077215, 0.2640359, -0...|
 Mogul|[-0.12976612, -0.41364384, 0.64921045, -1.3301969, -0.46330756, -0.7565513, 0.34162408, 0.6580047...|
+------------+----------------------------------------------------------------------------------------------------+
only showing top 20 rows

### Chunk Embeddings

This annotator utilizes `WordEmbeddings` or `BertEmbeddings` to generate chunk embeddings from either `TextMatcher`, `RegexMatcher`, `Chunker`, `NGramGenerator`, or `NerConverter` outputs.

`setPoolingStrategy`: Choose how you would like to aggregate Word Embeddings to Sentence Embeddings: `AVERAGE` or `SUM`

In [0]:
news_df.take(3)

Out[60]: [Row(category='Business', text="Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul."),
 Row(category='Sci/Tech', text=' TORONTO, Canada A second team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for privately funded suborbital space flight, has officially announced the first launch date for its manned rocket.'),
 Row(category='Sci/Tech', text=' A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.')]

In [0]:
entities = ['parent firm', 'economy', 'amino acids']
with open ('entities.txt', 'w') as f:
    for i in entities:
        f.write(i+'\n')
        
dbutils.fs.cp("file:/databricks/driver/entities.txt", "dbfs:/")

entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("entities")\
    .setEntities("file:/databricks/driver/entities.txt")\
    .setCaseSensitive(False)\
    .setEntityValue('entities')

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 entity_extractor
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(10))

result.select('entities.result').take(3)


Out[62]: [Row(result=['parent firm']), Row(result=[]), Row(result=['amino acids'])]

In [0]:
chunk_embeddings = ChunkEmbeddings() \
            .setInputCols(["entities", "embeddings"]) \
            .setOutputCol("chunk_embeddings") \
            .setPoolingStrategy("AVERAGE")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 entity_extractor,
 glove_embeddings,
 chunk_embeddings
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [0]:
result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('entities.result', 'chunk_embeddings.embeddings')).alias("cols")) \
.select(F.expr("cols['0']").alias("entities"),
        F.expr("cols['1']").alias("glove_embeddings"))

result_df.show(truncate=100)

+-----------+----------------------------------------------------------------------------------------------------+
 entities| glove_embeddings|
+-----------+----------------------------------------------------------------------------------------------------+
parent firm|[0.45683652, -0.105479494, -0.34525, -0.143924, -0.192452, -0.33616, -0.22334, -0.208185, -0.3673...|
amino acids|[-0.3861, 0.054408997, -0.287795, -0.33318, 0.375065, -0.185539, -0.330525, -0.214415, -0.73892, ...|
+-----------+----------------------------------------------------------------------------------------------------+

In [0]:
news_df.show()

+--------+--------------------+
category| text|
+--------+--------------------+
Business|Unions representi...|
Sci/Tech| TORONTO, Canada ...|
Sci/Tech| A company founde...|
Sci/Tech| It's barely dawn...|
Sci/Tech| Southern Califor...|
Sci/Tech|"The British Depa...|
Sci/Tech|"confessed author...|
Sci/Tech|\\FOAF/LOAF and ...|
Sci/Tech|"Wiltshire Police...|
Sci/Tech|In its first two ...|
Sci/Tech| A group of techn...|
Sci/Tech| Apple Computer I...|
Sci/Tech| Free Record Shop...|
Sci/Tech|A giant 100km col...|
Sci/Tech|"Dolphin groups, ...|
Sci/Tech|Tyrannosaurus rex...|
Sci/Tech| Scientists have...|
Sci/Tech| ESAs Mars Expre...|
Sci/Tech|When did life beg...|
Sci/Tech|update Earnings p...|
+--------+--------------------+
only showing top 20 rows

### UniversalSentenceEncoder

The Universal Sentence Encoder encodes text into high dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language tasks.

In [0]:
# no need for token columns 
use_embeddings = UniversalSentenceEncoder.pretrained('tfhub_use').\
  setInputCols(["document"]).\
  setOutputCol("sentence_embeddings")


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 use_embeddings
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('document.result', 'sentence_embeddings.embeddings')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("USE_embeddings"))

result_df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 document| USE_embeddings|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
Unions representing workers at Turner Newall say they are 'disappointed' after talks with stric...|[0.01299754, 0.019844763, -0.02462647, 0.039759073, -0.04424647, 0.01319757, 0.0786744, -0.053629...|
 TORONTO, Canada A second team of rocketeers competing for the #36;10 million Ansari X Prize,...|[0.0019999044, 0.051844064, -0.0440291, -5.932768E-4, -0.038505998, -0.02727945, 0.069404684, -0....|
 A company founded by a chemistry researcher at the University of Louisville won a grant to devel...|[0.038641855, 0.023220802, -0.0040162755, 0.0719947, 0.02727973, -0.05895184, -0.001953841, 0.002...|
 It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures an...|[0.07239491, 0.065592386, -0.015342037, -0.02381427, 0.013229337, -0.046775304, 0.009602194, 0.02...|
 Southern California's smog fighting agency went after emissions of the bovine variety Friday, ad...|[-0.034784008, -0.0437179, -8.897013E-4, 0.06168911, 0.033439934, 0.021937974, 0.053599276, -0.05...|
"The British Department for Education and Skills (DfES) recently launched a ""Music Manifesto"" c...|[-0.022449756, -0.005116286, -0.011258117, 0.037045613, -0.0015617298, -0.046843972, 0.08016194, ...|
"confessed author of the Netsky and Sasser viruses, is responsible for 70 percent of virus infect...|[4.522043E-4, -0.044989046, -0.027128313, 0.057110913, 0.04432687, -0.039309442, -0.00496823, -0....|
\\FOAF/LOAF and bloom filters have a lot of interesting properties for social\network and whitel...|[0.045447677, -0.0673045, 0.047636587, -0.020948866, -0.025080303, 0.03516154, 0.0047589587, 0.03...|
 "Wiltshire Police warns about ""phishing"" after its fraud squad chief was targeted."|[-0.0051346202, -0.07799961, -0.03284713, -0.010246719, 0.037006848, -0.0535042, 0.009521304, -0....|
In its first two years, the UK's dedicated card fraud unit, has recovered 36,000 stolen cards and...|[0.007892213, -0.070118815, -0.028799068, -0.018356517, 0.02493721, -0.07232398, 0.085445836, 0.0...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+

## Loading Models from local

In [0]:
glove_embeddings = WordEmbeddingsModel.load('/databricks/driver/glove_100d_en').\
  setInputCols(["document", 'token']).\
  setOutputCol("glove_embeddings")
    

## Getting Sentence Embeddings from word embeddings

In [0]:
glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE") # or SUM


nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 glove_embeddings,
 embeddingsSentence
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('document.result', 'sentence_embeddings.embeddings')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentence_embeddings"))

result_df.show(truncate=100)



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][OK!]
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 document| sentence_embeddings|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
Unions representing workers at Turner Newall say they are 'disappointed' after talks with stric...|[0.06131713, 0.08552574, 0.1340617, -0.22403374, -0.23798925, -0.159222, -0.21079227, 0.07760903,...|
 TORONTO, Canada A second team of rocketeers competing for the #36;10 million Ansari X Prize,...|[-0.05313636, 0.100133374, 0.19096054, -0.01998518, 0.32574704, -0.06335781, 0.1789568, 0.2931641...|
 A company founded by a chemistry researcher at the University of Louisville won a grant to devel...|[0.021042928, 0.074208066, 0.0723595, -0.09070449, 0.2993206, 0.04752573, -0.060713578, 0.155145,...|
 It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures an...|[-0.22619276, 0.18458064, 0.3589564, -0.3349514, 0.08098127, 0.09799263, -0.049265776, 0.25009856...|
 Southern California's smog fighting agency went after emissions of the bovine variety Friday, ad...|[-0.06140215, -0.040404316, 0.44608322, -0.18327837, 0.019998113, -0.09348459, -0.17261569, 0.177...|
"The British Department for Education and Skills (DfES) recently launched a ""Music Manifesto"" c...|[-0.01894932, 0.067353465, 0.011856217, -0.23963752, 0.30282092, 0.01841911, 0.029716793, 0.16109...|
"confessed author of the Netsky and Sasser viruses, is responsible for 70 percent of virus infect...|[0.013585807, 0.0013871301, 0.19375019, -0.25756782, 0.1557032, 0.101828136, 0.12016087, 0.031324...|
\\FOAF/LOAF and bloom filters have a lot of interesting properties for social\network and whitel...|[-0.13172166, 0.16039874, 0.26770115, -0.21353802, 0.094076656, 0.123355016, -0.1721528, 0.098089...|
 "Wiltshire Police warns about ""phishing"" after its fraud squad chief was targeted."|[-0.014898933, -0.1829025, 0.17554195, -0.355318, -0.013315629, 0.18482018, -0.0075285546, 0.0961...|
In its first two years, the UK's dedicated card fraud unit, has recovered 36,000 stolen cards and...|[0.0010903521, 0.17995015, 0.18142067, -0.24167466, 0.19849405, 0.19571333, 0.11464099, 0.1724940...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+

### Cosine similarity between two embeddings (sentence similarity)

In [0]:
from scipy.spatial import distance

import numpy as np

v1 = result_df.select('sentence_embeddings').take(2)[0][0]

v2 = result_df.select('sentence_embeddings').take(2)[0][0]

1 - distance.cosine(np.array(v1), np.array(v2))

Out[70]: 1.0

In [0]:
v2 = result_df.select('sentence_embeddings').take(2)[0][0]

1 - distance.cosine(np.array(v1), np.array(v2))

Out[71]: 1.0

## NERDL Model

### Public NER (CoNLL 2003)

<p><strong>Named-Entity recognition</strong> is a well-known technique in information extraction it is also known as&nbsp;<strong>entity identification</strong>,&nbsp;<strong>entity chunking</strong>&nbsp;and&nbsp;<strong>entity extraction.</strong>&nbsp;Knowing the relevant tags for each article help in automatically categorizing the articles in defined hierarchies and enable smooth content discovery.

Entities

``` PERSON, LOCATION, ORGANIZATION, MISC ```

In [0]:
public_ner = NerDLModel.pretrained("ner_dl_bert", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[ | ][ / ][ — ][ \ ][OK!]

In [0]:
public_ner.getClasses()

Out[73]: ['O', 'I-PER', 'I-MISC', 'I-ORG', 'I-LOC', 'B-MISC', 'B-LOC', 'B-ORG']

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# ner_dl_bert model is trained with bert_embeddings. So we use the same embeddings in the pipeline
public_ner = NerDLModel.pretrained("ner_dl_bert", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 bert_embeddings,
 public_ner
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)


ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[ | ][OK!]

In [0]:
result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)



+------------+---------+
 token|ner_label|
+------------+---------+
 Unions| O|
representing| O|
 workers| O|
 at| O|
 Turner| I-ORG|
 Newall| I-ORG|
 say| O|
 they| O|
 are| O|
 '| O|
disappointed| O|
 '| O|
 after| O|
 talks| O|
 with| O|
 stricken| O|
 parent| O|
 firm| O|
 Federal| I-ORG|
 Mogul| I-ORG|
 .| null|
 TORONTO| I-LOC|
 ,| O|
 Canada| I-LOC|
 A| O|
 second| O|
 team| O|
 of| O|
 rocketeers| O|
 competing| O|
 for| O|
 the| O|
 #36;10| O|
 million| O|
 Ansari| I-MISC|
 X| I-MISC|
 Prize| I-MISC|
 ,| O|
 a| O|
 contest| O|
 for| O|
 privately| O|
 funded| O|
 suborbital| O|
 space| O|
 flight| O|
 ,| O|
 has| O|
 officially| O|
 announced| O|
+------------+---------+
only showing top 50 rows

### NerDL OntoNotes 100D

This pipeline is based on&nbsp;<strong>NerDLApproach</strong> annotator with <strong>Char CNN - BiLSTM</strong> and <strong>GloVe Embeddings</strong> on the <strong>OntoNotes</strong> corpus and supports the identification of 18 entities.</p><p>Following NER types are supported in this pipeline:</p><table><thead><tr><th>Type</th><th>Description</th></tr></thead><tbody><tr><td><code>PERSON</code></td><td>People, including fictional.</td></tr><tr><td><code>NORP</code></td><td>Nationalities or religious or political groups.</td></tr><tr><td><code>FAC</code></td><td>Buildings, airports, highways, bridges, etc.</td></tr><tr><td><code>ORG</code></td><td>Companies, agencies, institutions, etc.</td></tr><tr><td><code>GPE</code></td><td>Countries, cities, states.</td></tr><tr><td><code>LOC</code></td><td>Non-GPE locations, mountain ranges, bodies of water.</td></tr><tr><td><code>PRODUCT</code></td><td>Objects, vehicles, foods, etc. (Not services.)</td></tr><tr><td><code>EVENT</code></td><td>Named hurricanes, battles, wars, sports events, etc.</td></tr><tr><td><code>WORK_OF_ART</code></td><td>Titles of books, songs, etc.</td></tr><tr><td><code>LAW</code></td><td>Named documents made into laws.</td></tr><tr><td><code>LANGUAGE</code></td><td>Any named language.</td></tr><tr><td><code>DATE</code></td><td>Absolute or relative dates or periods.</td></tr><tr><td><code>TIME</code></td><td>Times smaller than a day.</td></tr><tr><td><code>PERCENT</code></td><td>Percentage, including &rdquo;%&ldquo;.</td></tr><tr><td><code>MONEY</code></td><td>Monetary values, including unit.</td></tr><tr><td><code>QUANTITY</code></td><td>Measurements, as of weight or distance.</td></tr><tr><td><code>ORDINAL</code></td><td>&ldquo;first&rdquo;, &ldquo;second&rdquo;, etc.</td></tr><tr><td><code>CARDINAL</code></td><td>Numerals that do not fall under another type.</td></tr></tbody></table>

Entities

``` 'CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART' ```

In [0]:
onto_ner = NerDLModel.pretrained("onto_100", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 glove_embeddings,
 onto_ner
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

light_model =  LightPipeline(pipelineModel)

onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[ | ][ / ][OK!]

In [0]:
result = pipelineModel.transform(news_df.limit(10))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)


+------------+-------------+
 token| ner_label|
+------------+-------------+
 Unions| O|
representing| O|
 workers| O|
 at| O|
 Turner| B-ORG|
 Newall| I-ORG|
 say| O|
 they| O|
 are| O|
 '| O|
disappointed| O|
 '| O|
 after| O|
 talks| O|
 with| O|
 stricken| O|
 parent| O|
 firm| O|
 Federal| B-ORG|
 Mogul| I-ORG|
 .| O|
 TORONTO| B-GPE|
 ,| O|
 Canada| B-GPE|
 A| O|
 second| B-ORDINAL|
 team| O|
 of| O|
 rocketeers| O|
 competing| O|
 for| O|
 the| O|
 #36;10| B-CARDINAL|
 million| I-CARDINAL|
 Ansari|B-WORK_OF_ART|
 X|I-WORK_OF_ART|
 Prize|I-WORK_OF_ART|
 ,| O|
 a| O|
 contest| O|
 for| O|
 privately| O|
 funded| O|
 suborbital| O|
 space| O|
 flight| O|
 ,| O|
 has| O|
 officially| O|
 announced| O|
+------------+-------------+
only showing top 50 rows

## Highlight the entities

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/utils/ner_highlighter.py

# Add the path to system, local or mounted S3 bucket, e.g. /dbfs/mnt/<path_to_bucket>
sys.path.append('/databricks/driver/')
sys.path.append('/databricks/driver/ner_highlighter.py')

#dbutils.fs.cp("file:/databricks/driver/ner_highlighter.py", "dbfs:/")

In [0]:
light_data = light_model.annotate('Unions representing workers at Turner Newall say they are disappointed after talks with stricken parent firm Federal Mogul in California.')

light_data

Out[149]: {'document': ['Unions representing workers at Turner Newall say they are disappointed after talks with stricken parent firm Federal Mogul in California.'],
 'token': ['Unions',
 'representing',
 'workers',
 'at',
 'Turner',
 'Newall',
 'say',
 'they',
 'are',
 'disappointed',
 'after',
 'talks',
 'with',
 'stricken',
 'parent',
 'firm',
 'Federal',
 'Mogul',
 'in',
 'California',
 '.'],
 'embeddings': ['Unions',
 'representing',
 'workers',
 'at',
 'Turner',
 'Newall',
 'say',
 'they',
 'are',
 'disappointed',
 'after',
 'talks',
 'with',
 'stricken',
 'parent',
 'firm',
 'Federal',
 'Mogul',
 'in',
 'California',
 '.'],
 'ner': ['O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O',
 'B-GPE',
 'O']}

In [0]:
displayHTML(ner_highlighter.token_highlighter(light_data))

Unions representing workers at Turner Newall say they are disappointed after talks with stricken parent firm Federal Mogul in California . Color codes: GPE , ORG ,

### NER with Bert (CoNLL 2003)

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained('bert_base_cased')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

onto_ner_bert = NerDLModel.pretrained("ner_dl_bert", 'en') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

onto_ner_bert.getStorageRef()

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 bert_embeddings,
 onto_ner_bert
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[ | ][ / ][OK!]

In [0]:
# fullAnnotate in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('Peter Parker is a nice persn and lives in New York. Bruce Wayne is also a nice guy and lives in Gotham City.')

list(zip(light_result['token'], light_result['ner']))

### Getting the NER chunks with NER Converter

In [0]:

ner_converter = NerConverter() \
  .setInputCols(["document", "token", "ner"]) \
  .setOutputCol("ner_chunk")


nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 bert_embeddings,
 onto_ner_bert,
 ner_converter
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(news_df.limit(10))



In [0]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-------------------------------------------+---------+
chunk |ner_label|
+-------------------------------------------+---------+
Turner Newall |ORG |
Federal Mogul |ORG |
TORONTO |LOC |
Canada |LOC |
Ansari X Prize |MISC |
University of Louisville |ORG |
Mike Fitzpatrick |PER |
Southern California's |LOC |
British Department for Education and Skills|ORG |
DfES |ORG |
Netsky |MISC |
Sasser |MISC |
Sophos |ORG |
Jaschan |PER |
Germany |LOC |
Netsky |MISC |
Sasser |MISC |
GPG/OpenPGP |MISC |
FOAF |ORG |
PGP |ORG |
+-------------------------------------------+---------+
only showing top 20 rows

In [0]:

# fullAnnotate in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.fullAnnotate('Peter Parker is a nice persn and lives in New York. Bruce Wayne is also a nice guy and lives in Gotham City center.')


chunks = []
entities = []

for n in light_result[0]['ner_chunk']:
        
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities})

df

,chunks,entities
0,Peter Parker,PER
1,New York,LOC
2,Bruce Wayne,PER
3,Gotham City,LOC


End of Notebook # 3